In [1]:
import sqlalchemy as sa
import urllib
import pandas as pd
import pyodbc
import time
import glob
from datetime import datetime, timedelta
from os import listdir
from os.path import isfile, join
from  tqdm import tqdm
import os
import warnings
import datetime
import numpy as np
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 41)

connection_string = (
    'Driver={ODBC Driver 17 for SQL Server};'
    'SERVER=118.69.201.34;'
    'Database=FLC_SHOP_SALES_DATAWAREHOUSE;'
    'UID=ecom_user;'
    'PWD=Ec0m@12345;'
    'Trusted_Connection=no;'
)

connection_uri = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(str(connection_string))}"
engine = sa.create_engine(connection_uri, fast_executemany=True)
def getEngine(engine_name):
    connection = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(engine_name)}"
    engine = sa.create_engine(connection, fast_executemany=True)
    return engine
    
### Connect to DataMart Server
mart_server = '118.69.201.34'
mart_database = 'FLC_SHOP_SALES_DATAWAREHOUSE'
mart_username = 'ecom_user'
mart_password = 'Ec0m@12345'
connection_mart = sa.engine.URL.create(
    "mssql+pyodbc",
    username=mart_username,
    password=mart_password,
    host=mart_server,
    database=mart_database,
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "autocommit": "True",
    },
)
mart_engine = sa.create_engine(connection_mart)

# SaleMart = ('DRIVER={ODBC Driver 17 for SQL Server}'
#             ';SERVER=' + mart_server +
#             ';DATABASE=' + mart_database +
#             ';UID=' + mart_username +
#             ';PWD=' + mart_password)


### Connect to Azure server
azr_server = "ssd-synapse-prod.sql.azuresynapse.net"
azr_database = 'frtssdsqlpoolprod'
azr_username = 'frtcid'
azr_password = 'Frt2022Cid@#123'
        
connection_url = sa.engine.URL.create(
    "mssql+pyodbc",
    username=azr_username,
    password=azr_password,
    host=azr_server,
    database=azr_database,
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "autocommit": "True",
    },
)

azr_engine = sa.create_engine(connection_url)

# Cập nhập dữ liệu LCNB

In [14]:
df_lcnb = pd.read_csv(r'D:\LCNB_H\2023\202306.csv', sep='\t', encoding = 'utf-16', on_bad_lines='skip')
columns = ['Ngày Tạo HO', 'Người Tạo','Số Phiếu YCĐC','Mã SP','Số Imei','Số Lượng Xuất','Số Lượng Nhập','Số Điều Chuyển Xuất','Tình Trạng Phiếu Xuất','Số Điều Chuyển Nhập','Tình Trạng Phiếu Nhập','Mã Shop Xuất','Mã Shop Nhập','Ngày Xuất Thủ Kho','Ngày Nhập Hàng']
df_lcnb_filter = pd.DataFrame(df_lcnb, columns=columns)
df_lcnb_filter['Ngày Tạo HO']=df_lcnb_filter['Ngày Tạo HO'].str.replace("'", '')
df_lcnb_filter['Người Tạo']=df_lcnb_filter['Người Tạo'].str.replace("'", '')
df_lcnb_filter['Số Imei']=df_lcnb_filter['Số Imei'].str.replace("'", '')
df_lcnb_filter['Ngày Xuất Thủ Kho']=df_lcnb_filter['Ngày Xuất Thủ Kho'].str.replace("'", '')
df_lcnb_filter['Ngày Nhập Hàng']=df_lcnb_filter['Ngày Nhập Hàng'].str.replace("'", '')
df_lcnb_filter['Mã Shop Xuất'] = df_lcnb_filter['Mã Shop Xuất'].str.replace(" ", '')
# df_lcnb_filter['Mã Shop Xuất'] = df_lcnb_filter['Mã Shop Xuất'].replace(['inf', 'NaN'], np.nan)
# df_lcnb_filter['Mã Shop Nhập'] = df_lcnb_filter['Mã Shop Nhập'].replace(['inf', 'NaN'], np.nan)
# df_lcnb_filter['Mã Shop Xuất'] = df_lcnb_filter['Mã Shop Xuất'].astype(float).astype('Int64')
# df_lcnb_filter['Mã Shop Nhập'] = df_lcnb_filter['Mã Shop Nhập'].astype(float).astype('Int64')
# df_lcnb_filter['Số Phiếu YCĐC'] = df_lcnb_filter['Số Phiếu YCĐC'].astype(float).astype('Int64')
df_lcnb_filter['Số Điều Chuyển Xuất'] = df_lcnb_filter['Số Điều Chuyển Xuất'].astype(float).astype('Int64')
df_lcnb_filter['Số Điều Chuyển Nhập'] = df_lcnb_filter['Số Điều Chuyển Nhập'].astype(float).astype('Int64')
df_lcnb_filter.head(3)

,Ngày Tạo HO,Người Tạo,Số Phiếu YCĐC,Mã SP,Số Imei,Số Lượng Xuất,Số Lượng Nhập,Số Điều Chuyển Xuất,Tình Trạng Phiếu Xuất,Số Điều Chuyển Nhập,Tình Trạng Phiếu Nhập,Mã Shop Xuất,Mã Shop Nhập,Ngày Xuất Thủ Kho,Ngày Nhập Hàng
0,19/06/2023 16:28:35,38602 - Văn Hoàng Kha,11688381,'00839719,351040494740476,1.0,1.0,21854731,NaN,21861294,NaN,NaN,30871,19/06/2023 19:48:42,21/06/2023 09:46:53
1,24/06/2023 10:41:30,22061 - Đặng Thị Dung,11703271,'00832927,SMCF2N6339X,1.0,1.0,21887503,Hoàn tất,21888117,Hoàn tất,NaN,30220,24/06/2023 11:41:48,26/06/2023 16:21:33
2,29/06/2023 13:40:32,37805 - Lưu Hữu Kiên,11722321,'11018150SK24RH9D9HG,SK24RH9D9HG,1.0,1.0,21923668,Hoàn tất,21923672,Hoàn tất,NaN,11000,29/06/2023 13:40:47,29/06/2023 13:40:48


In [15]:
df_lcnb_filter.to_sql("CID_WRT_ITR", engine, schema="dbo", if_exists="append", index=False)

-1

# Kiểm tra dữ liệu doanh thu, số hóa đơn, số khách mua hàng tại shop trong ngày

In [12]:
# Đường dẫn đến thư mục chứa các file CSV
folder_path = r"D:\TRANSACTION_ICT\2023"

# Danh sách các file CSV trong thư mục
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]

# Đọc và merge các file CSV
merged_df = pd.DataFrame()
for file in csv_files:
    file_path = os.path.join(folder_path, file)
    df = pd.read_csv(file_path,encoding='utf-16', delimiter='\t')
    merged_df = merged_df.append(df, ignore_index=True)

In [13]:
df_group_sum_amt = merged_df.groupby(['Ngày Cập Nhật Doanh Thu', 'Mã Shop'], as_index = False)['Thành Tiền'].sum()
group_sum_amt = pd.DataFrame(df_group_sum_amt)
df_group_count_inv = merged_df.groupby(['Ngày Cập Nhật Doanh Thu', 'Mã Shop'], as_index = False)['Số Đơn Hàng'].nunique()
group_count_inv = pd.DataFrame(df_group_count_inv)
df_group_count_cus = merged_df.groupby(['Ngày Cập Nhật Doanh Thu', 'Mã Shop'], as_index = False)['Mã Khách Hàng'].nunique()
group_count_cus = pd.DataFrame(df_group_count_cus)

In [14]:
result = pd.merge(pd.merge(group_sum_amt, group_count_inv, on=['Ngày Cập Nhật Doanh Thu', 'Mã Shop']), group_count_cus,on=['Ngày Cập Nhật Doanh Thu', 'Mã Shop'])
result.rename(columns={
                        result.columns[0]: 'DATE_KEY', 
                        result.columns[1]: 'SHOP_CODE', 
                        result.columns[2]: 'SUM_AMT', 
                        result.columns[3]: 'C_INV', 
                        result.columns[4]: 'C_CUS'}, inplace = True)
result['DATE_KEY'] = result['DATE_KEY'].apply(lambda x: datetime.datetime.strptime(x, "'%d/%m/%Y").strftime('%Y%m%d'))
result['DATE_KEY'] = result['DATE_KEY'].astype('int64')
result

,DATE_KEY,SHOP_CODE,SUM_AMT,C_INV,C_CUS
0,20230101,30101,2.138569e+08,25,22
1,20230101,30103,3.427787e+08,38,33
2,20230101,30104,2.846800e+07,12,8
3,20230101,30105,8.654120e+07,17,9
4,20230101,30106,1.581674e+08,21,16
...,...,...,...,...,...
143921,20230531,40704,2.762900e+07,3,2
143922,20230531,40801,2.136222e+08,29,15
143923,20230531,40805,2.346710e+08,41,19
143924,20230531,40806,2.413838e+08,23,14


In [10]:
query_transactions_ict = f"""SELECT a.DATE_KEY, b.SHOP_CODE, 
SUM(CASE WHEN LEFT(TRANSACTION_TYPE,6) = 'Return' THEN (a.LIST_AMT_AF_VAT*-1) ELSE a.LIST_AMT_AF_VAT END) SUM_AMT, 
COUNT(DISTINCT a.INVOICE_HEADER) C_INV, COUNT(DISTINCT a.CUSTOMER_KEY) C_CUS
FROM dbo.VF_CID_SALES_TRANSACTION_ICT a
LEFT JOIN dbo.VD_CID_SHOP_ICT b
ON a.SHOP_KEY = b.SHOP_KEY
WHERE 1 = 1
AND a.W_DELETE_FLG = 'N'
AND a.DATE_KEY >= 20230101
GROUP BY a.DATE_KEY, b.SHOP_CODE
ORDER BY a.DATE_KEY, b.SHOP_CODE
"""
query_shop_lc_missing = f"""SELECT DISTINCT a.SHOP_KEY, b.SHOP_CODE
FROM dbo.VF_CID_SALES_TRANSACTION_FLC a
LEFT JOIN dbo.VD_CID_SHOP_LC b
ON a.SHOP_KEY = b.SHOP_KEY
WHERE b.SHOP_CODE IS NULL
ORDER BY b.SHOP_CODE"""

query_shop_ict_missing = f"""SELECT DISTINCT a.SHOP_KEY, b.SHOP_CODE
FROM dbo.VF_CID_SALES_TRANSACTION_ICT a
LEFT JOIN dbo.VD_CID_SHOP_ICT b
ON a.SHOP_KEY = b.SHOP_KEY
WHERE b.SHOP_CODE IS NULL
ORDER BY b.SHOP_CODE"""

transactions_ict = pd.read_sql_query(query_transactions_ict,azr_engine)
shop_lc_missing = pd.read_sql_query(query_shop_lc_missing,azr_engine)
shop_ict_missing = pd.read_sql_query(query_shop_ict_missing,azr_engine)
transactions_ict['SHOP_CODE'] = transactions_ict['SHOP_CODE'].astype('int64')
transactions_ict

,DATE_KEY,SHOP_CODE,SUM_AMT,C_INV,C_CUS
0,20230101,30101,2.138569e+08,25,22
1,20230101,30103,3.427787e+08,38,33
2,20230101,30104,2.846800e+07,12,8
3,20230101,30105,8.654120e+07,17,9
4,20230101,30106,1.581674e+08,21,16
...,...,...,...,...,...
147136,20230704,40704,4.077050e+06,3,3
147137,20230704,40801,1.106634e+08,13,10
147138,20230704,40805,1.986132e+08,19,12
147139,20230704,40806,5.974000e+06,4,4


In [15]:
mapping_azure_34 = pd.merge(transactions_ict, result, on=['DATE_KEY','SHOP_CODE'], how='inner', suffixes=['_AZURE', '_BI'])
mapping_azure_34

,DATE_KEY,SHOP_CODE,SUM_AMT_AZURE,C_INV_AZURE,C_CUS_AZURE,SUM_AMT_BI,C_INV_BI,C_CUS_BI
0,20230101,30101,2.138569e+08,25,22,2.138569e+08,25,22
1,20230101,30103,3.427787e+08,38,33,3.427787e+08,38,33
2,20230101,30104,2.846800e+07,12,8,2.846800e+07,12,8
3,20230101,30105,8.654120e+07,17,9,8.654120e+07,17,9
4,20230101,30106,1.581674e+08,21,16,1.581674e+08,21,16
...,...,...,...,...,...,...,...,...
143921,20230630,40704,9.800000e+04,1,1,9.800000e+04,1,1
143922,20230630,40801,2.285070e+08,22,16,2.285070e+08,22,16
143923,20230630,40805,1.740114e+08,21,16,1.740114e+08,21,16
143924,20230630,40806,1.037610e+08,17,12,1.037610e+08,17,12


In [16]:
mapping_azure_34['DIFF_AMT'] = abs(mapping_azure_34['SUM_AMT_AZURE'] - mapping_azure_34['SUM_AMT_BI'])
mapping_azure_34['DIFF_INVOICE'] = abs(mapping_azure_34['C_INV_AZURE'] - mapping_azure_34['C_INV_BI'])
mapping_azure_34['DIFF_CUS'] = abs(mapping_azure_34['C_CUS_AZURE'] - mapping_azure_34['C_CUS_BI'])
pd.set_option('display.float_format', '{:.2f}'.format)
mapping_azure_34['PERCENT_AMT'] = (mapping_azure_34['DIFF_AMT']/mapping_azure_34['SUM_AMT_BI'])*100
filtered_mapping = mapping_azure_34.loc[mapping_azure_34['PERCENT_AMT'] > 50]
filtered_mapping.sort_values(by = 'PERCENT_AMT', ascending=False)

,DATE_KEY,SHOP_CODE,SUM_AMT_AZURE,C_INV_AZURE,C_CUS_AZURE,SUM_AMT_BI,C_INV_BI,C_CUS_BI,DIFF_AMT,DIFF_INVOICE,DIFF_CUS,PERCENT_AMT
67874,20230329,30157,-671409.42,5,5,1943991.43,5,5,2615400.86,0,0,134.54
68523,20230329,31373,318633.57,15,12,10246394.16,15,12,9927760.59,0,0,96.89


In [17]:
df = pd.read_csv(r"D:\TRANSACTION_ICT\2023\03-2023.csv",encoding='utf-16', delimiter='\t')

In [19]:
df_filtered = df.loc[df['Mã Shop'] == 30157]
df_filtered.to_excel(r"D:\TRANSACTION_ICT\CHECK_30157_!.xlsx", index = False)

In [ ]:
# result.to_excel(r"D:\TRANSACTION_ICT\TEST_DT.xlsx")

## END

# Kiểm tra dữ liệu khách hàng và import vào server 34:

In [9]:
# query_sales_trans_ict = f"""SELECT DATE_KEY, COUNT(*) COUNT_LINE, 'trucvhc' AS INSERT_BY
# FROM dbo.VF_CID_SALES_TRANSACTION_ICT
# GROUP BY DATE_KEY
# ORDER BY DATE_KEY"""
# trans_ict = pd.read_sql_query(query_sales_trans_ict,azr_engine)
# query_shop_lc = f"""SELECT DATE_KEY, COUNT(*) COUNT_LINE, 'trucvhc' AS INSERT_BY
# FROM dbo.VF_CID_SALES_TRANSACTION_ICT
# GROUP BY DATE_KEY
# ORDER BY DATE_KEY"""
query_customer_34 = f"""SELECT CAST(DATEADD(DAY,1,MAX(CUSTOMER_CREATED_DT)) AS DATE) _MAXD
FROM dbo.CID_CUSTOMER_FLC"""
customer_34 = pd.read_sql_query(query_customer_34,mart_engine)
query_customer_azure = f"""SELECT CUSTOMER_KEY, CUSTOMER_CODE,
CASE WHEN CUSTOMER_NAME = 'Unspecified' THEN NULL ELSE CUSTOMER_NAME END AS CUSTOMER_NAME, 
CASE WHEN CUSTOMER_ADDRESS = 'Unspecified' THEN NULL ELSE CUSTOMER_ADDRESS END AS CUSTOMER_ADDRESS, 
CUSTOMER_PHONE, CUSTOMER_TYPE, CUSTOMER_PRIVATE_TYPE, CUSTOMER_CREATED_DT
FROM dbo.VD_CID_CUSTOMER_FLC
WHERE CUSTOMER_CREATED_DT >= '{customer_34['_MAXD'][0]}'
AND CUSTOMER_CREATED_DT < CAST(GETDATE() AS DATE)
ORDER BY CUSTOMER_CREATED_DT
"""
customer_azure = pd.read_sql_query(query_customer_azure,azr_engine)
customer_azure

,CUSTOMER_KEY,CUSTOMER_CODE,CUSTOMER_NAME,CUSTOMER_ADDRESS,CUSTOMER_PHONE,CUSTOMER_TYPE,CUSTOMER_PRIVATE_TYPE,CUSTOMER_CREATED_DT


In [8]:
customer_azure.to_sql("CID_CUSTOMER_FLC", engine, schema="dbo", if_exists="append", index=False)

-1

# END

# Kiểm tra và lưu trữ dữ liệu tích tiêu điểm FLC:

In [78]:
check_point_use = f"""SELECT *
FROM dbo.CID_LOYALTY_FS
WHERE TransactionPointType IN ('205','202')
AND OrderCode IS NULL
ORDER BY TransactionDate"""
trans_point_use = pd.read_sql_query(check_point_use,engine)
trans_point_use

In [80]:
merged_df = pd.merge(trans_point_use, df_check, on=['phonenumber', 'TransactionDate','transactionPoint','ActualPoint','TransactionAmount','TransactionPointType'], how='inner')
filtered_df = merged_df[merged_df['OrderCode_y'].notna()]
filtered_df = filtered_df.drop(filtered_df.columns[[6]], axis=1).drop_duplicates()
filtered_df.rename(columns={filtered_df.columns[6]: 'OrderCode'}, inplace = True)
filtered_df

,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode


,phonenumber,transactionPoint,ActualPoint,TransactionAmount,TransactionPointType,TransactionDate,OrderCode


In [65]:
phone_arr = filtered_df['phonenumber'].values
phone_list = phone_arr.tolist()
date_arr = filtered_df['TransactionDate'].values
date_list = date_arr.tolist()
# Chuyển đổi danh sách các chuỗi thành một chuỗi đúng để sử dụng trong truy vấn SQL
phone = "'" + "','".join(phone_list) + "'"
date = "'" + "','".join(date_list) + "'"
replace_point_use = f"""SELECT *
FROM dbo.CID_LOYALTY_FS
WHERE phonenumber IN ({phone})
AND TransactionPointType IN ('202','205')
AND TransactionDate IN ({date})"""
rs_replace_point_use = pd.read_sql_query(replace_point_use,engine)
rs_replace_point_use